In [1]:
import numpy as np
import pandas as pd

In [2]:
train_df = pd.read_csv("data/train.csv")
test_df = pd.read_csv("data/test.csv")

In [3]:
y_train = train_df.target
X_train = train_df.drop(columns=["target"])

In [4]:
import re
def remove_digits(DF):
    for i in np.arange(len(DF)):
        DF[i] = re.sub(r"\d+", "", DF[i], flags=re.UNICODE)

In [5]:
X_train = X_train[['url', 'title']].apply(lambda x:x[0] + " " + x[1], axis=1)
remove_digits(X_train)

In [6]:
test_df.head(10)

,id,url,title
0,135309,www.kommersant.ru,Шестой кассационный суд в Самаре начнет работу...
1,135310,urexpert.online,"Что такое индексация алиментов, кем и в каких ..."
2,135311,imperimeha.ru,Женщинам | Империя Меха - Part 12
3,135312,national-porn.com,"Небритые, волосатые киски: Порно всех стран и ..."
4,135313,2gis.ru,67
5,135314,ryazan.gorodrabot.ru,"Вакансия Мерчендайзер визитный в Рязани, Групп..."
6,135315,m.fotosklad.ru,Смартфон Apple iPhone XR 64GB Черный A2105 EU ...
7,135316,sim-dealer.ru,Духовой шкаф Siemens HB337GYS0R - Sim-Dealer. ...
8,135317,krasnoyarsk.hh.ru,Вакансия Аналитик Департамента внедрения инфор...
9,135318,vsetop.org,Anomaly Defenders v1.0 – торрент


In [7]:
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english') + stopwords.words('russian'))

In [8]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(stop_words=stop_words)

In [9]:
X_train_vectorized = vectorizer.fit_transform(X_train)

In [10]:
from sklearn.model_selection import train_test_split
Data_train, Data_test, Ans_train, Ans_test = train_test_split(X_train_vectorized, y_train, test_size=0.25, random_state=1, shuffle=True)

In [11]:
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score, f1_score, make_scorer
linear_model = SGDClassifier(loss='modified_huber', class_weight="balanced")
linear_model.fit(Data_train, Ans_train)

SGDClassifier(alpha=0.0001, average=False, class_weight='balanced',
              early_stopping=False, epsilon=0.1, eta0=0.0, fit_intercept=True,
              l1_ratio=0.15, learning_rate='optimal', loss='modified_huber',
              max_iter=1000, n_iter_no_change=5, n_jobs=None, penalty='l2',
              power_t=0.5, random_state=None, shuffle=True, tol=0.001,
              validation_fraction=0.1, verbose=0, warm_start=False)

In [12]:
y_pred = linear_model.predict(Data_test)
print("Accuracy:", accuracy_score(y_pred, Ans_test))
print("F1 score:", f1_score(y_pred, Ans_test))

Accuracy: 0.9939103701076032
F1 score: 0.9755866319032946


In [13]:
linear_model = SGDClassifier(loss='modified_huber', class_weight="balanced")
linear_model.fit(X_train_vectorized, y_train)
X_test = test_df.drop(columns=[])
X_test = X_test[['url', 'title']].apply(lambda x:x[0] + " " + x[1], axis=1)
remove_digits(X_test)
X_test_vectorized = vectorizer.transform(X_test)

In [14]:
test_df["target"] = linear_model.predict(X_test_vectorized).astype(bool)
test_df[["id", "target"]].to_csv("SmirnovLC.csv", index=False)